# 📓 Notebook 02 – Outils et Visualisations DR5

> But : Ce notebook regroupe les outils interactifs développés pour explorer, diagnostiquer et affiner les spectres téléchargés du catalogue LAMOST DR5, sans relancer le pipeline complet.  
> Il est destiné à l’analyse exploratoire rapide, à la visualisation augmentée, et au debug scientifique.

<br/>

## ⚙️ Setup & Imports
L'environnement est initialisé dynamiquement avec détection de la racine du projet et ajout du dossier ``src/`` au ``sys.path``.  
On y importe les classes utilitaires ``AstroVisualizer`` et ``setup_project_env``.

In [ ]:
# --- Imports des librairies externes ---
import os
import sys
from IPython.display import display, Markdown

# --- Imports de la librairie "astrospectro" ---
from utils import setup_project_env
from tools.visualizer import AstroVisualizer
from pipeline.classifier import SpectralClassifier

# --- Initialisation ---
paths = setup_project_env()
visualizer = AstroVisualizer(paths)

print("\nSetup terminé. Les outils de visualisation sont prêts.")

#

## Tableau de Bord de l'État du Dataset

1. **Importe ``DatasetBuilder`` :** On réutilise l'outil conçu pour gérer les lots d'entraînement.
2. ``_list_available_fits()`` : On appelle cette méthode pour qu'elle scanne le dossier ``data/raw/`` et compte tous les fichiers ``.fits.gz`` téléchargés.
3. ``_load_trained_log()`` : On appelle cette méthode pour qu'elle lise le ``fichier trained_spectra.csv`` et compte combien de spectres ont déjà été marqués comme "_utilisés_".
4. **Calcul :** Une simple soustraction nous donne le nombre de spectres qui sont nouveaux et prêts pour la prochaine session d'entraînement.
5. **Affichage Markdown :** On utilise Markdown pour afficher les résultats de manière claire et professionnelle, directement dans la sortie du notebook.

In [ ]:
visualizer.display_dataset_dashboard()

#

## 🧠 Explorateur de Header FITS

Outil interactif permettant de charger dynamiquement un spectre ``.fits.gz`` et d’en afficher les métadonnées structurées.

### **Utilisation typique :**

- Vérifier la cohérence des champs : coordonnées, type d’objet, date, filtre, seeing…
- Déboguer un spectre problématique
- Détecter des valeurs aberrantes avant traitement massif

*Compatible avec les headers compressés grâce à ``astropy.io.fits.``*

In [ ]:
display(Markdown("## Explorateur de Header FITS"))
display(Markdown("Utilisez le menu déroulant pour sélectionner un spectre et afficher ses métadonnées complètes."))
visualizer.interactive_header_explorer()

#

## ⚗️ Tuning Interactif des Raies Spectrales

Permet d’ajuster en direct les paramètres de détection des raies spectrales :

- **Prominence** (hauteur minimale pour détecter un pic)
- **Fenêtre** (largeur du sliding window autour du pic)

**Objectif : tester visuellement les hyperparamètres avant traitement global du dataset.**

### L’interface comprend :

- un menu de sélection du spectre
- deux sliders ``ipwidgets``
- un tracé du spectre avec surlignement automatique des pics détectés

In [ ]:
display(Markdown("--- \n## Analyseur de Spectre Augmenté"))
display(Markdown(
    "Cet outil tout-en-un vous permet de visualiser un spectre, d'ajuster les "
    "paramètres de détection de pics en temps réel, et d'évaluer la qualité "
    "des données et de l'analyse."
))

from tools.visualizer import AstroVisualizer
visualizer = AstroVisualizer(paths)
# Cet appel unique crée maintenant l'interface complète
visualizer.interactive_peak_tuner()

#

## 🧼 Analyse des Features Nulles
Analyse de qualité des données extraites, avec graphique à barres des colonnes avec trop de valeurs nulles ``(0.0)``.

### Ce module permet de :

- Repérer les features peu ou non utilisées
- Nettoyer les colonnes inexploitables
- Diagnostiquer les erreurs d’extraction

In [ ]:
display(Markdown("--- \n## Analyse de la Qualité des Features"))
display(Markdown("Cet outil analyse le dernier fichier de features généré et montre le pourcentage de valeurs nulles pour chaque feature. C'est essentiel pour identifier les features peu informatives."))
visualizer.analyze_feature_zeros()

#

## 🌌 Carte de Couverture Céleste

Affiche une **projection Mollweide** des plans d’observation inclus dans les spectres téléchargés.

- Axe horizontal : Ascension Droite (RA)
- Axe vertical : Déclinaison (Dec)
- Couleur & taille : nombre de spectres par plan

💡 Permet de visualiser la densité des données acquises sur la voûte céleste.

In [ ]:
display(Markdown("--- \n## Carte de Couverture Céleste"))
display(Markdown("Cette carte montre la position des plans d'observation que tu as téléchargés. La taille et la couleur des points indiquent le nombre de spectres par plan."))
visualizer.plot_sky_coverage()

#

## 🔎 Inspecteur de Modèles Entraînés

Outil permettant d'explorer les modèles sauvegardés ``.pkl`` :

- Visualisation des hyperparamètres
- Affichage de la feature importance (triée)

**Très utile pour analyser la qualité du classifieur, l’importance des raies spectrales, et affiner le feature engineering.**

In [ ]:
display(Markdown("--- \n## Inspecteur de Modèles Entraînés"))
display(Markdown(
    "Utilisez le menu déroulant pour sélectionner un modèle `.pkl` sauvegardé. "
    "Cet outil affichera ses hyperparamètres et un graphique montrant l'importance de chaque feature "
    "pour la classification."
))

# Cet appel unique crée l'interface d'inspection
visualizer.interactive_model_inspector()

#

## 🔭 Comparateur de Spectres Interactif

Cet outil puissant vous permet de superposer plusieurs spectres sur un même graphique pour une analyse comparative détaillée. C'est un instrument essentiel pour :

-   **Comparer des Étoiles de Même Type :** Visualisez les variations subtiles entre plusieurs étoiles de type 'G', par exemple.
-   **Comparer des Types Différents :** Superposez un spectre de type 'A' et un de type 'M' pour voir de vos propres yeux les différences fondamentales dans leurs signatures spectrales.
-   **Analyser l'Évolution Temporelle :** Si vous avez plusieurs observations d'une même étoile variable, vous pouvez les superposer pour étudier son évolution.

### Utilisation

1.  **Sélection Multiple :** Cliquez sur les noms de fichiers dans la liste. Maintenez la touche `Ctrl` (ou `Cmd` sur Mac) pour sélectionner plusieurs fichiers individuellement, ou `Shift` pour sélectionner une plage continue.
2.  **Normalisation :** Cochez la case "Normaliser les spectres" (recommandé) pour ramener tous les spectres à une échelle comparable.
3.  **Décalage Vertical :** Utilisez le slider "Décalage Y" pour espacer verticalement les spectres afin de mieux les distinguer et éviter qu'ils ne se chevauchent.

In [ ]:
display(Markdown("--- \n## Comparateur de Spectres"))
display(Markdown("Sélectionnez plusieurs spectres (maintenez `Ctrl` ou `Shift`) pour les superposer. Ajustez le décalage pour mieux les distinguer."))
visualizer.interactive_spectra_comparator()

#

## Comparateur de normalisation de spectre (en développement)

*Cellule pour générer la figure de normalisation "Avant/Après"*

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import gzip
from astropy.io import fits

# --- Imports de tes propres modules ---
from pipeline.preprocessor import SpectraPreprocessor

# --- Configuration ---
# On s'assure que l'instance 'visualizer' existe (créée dans la cellule de setup)
if 'visualizer' in locals():
    # On choisit deux spectres au hasard parmi ceux disponibles
    if len(visualizer.available_spectra) >= 2:
        sample_paths = random.sample(visualizer.available_spectra, 2)
        
        # --- Chargement et traitement des deux spectres ---
        preprocessor = SpectraPreprocessor()
        spectra_data = []
        for path in sample_paths:
            full_path = os.path.join(visualizer.paths["RAW_DATA_DIR"], path)
            with gzip.open(full_path, 'rb') as f_gz:
                with fits.open(f_gz, memmap=False) as hdul:
                    wavelength, flux, _ = preprocessor.load_spectrum(hdul)
                    flux_norm = preprocessor.normalize_spectrum(flux)
                    spectra_data.append({
                        'wavelength': wavelength,
                        'flux_raw': flux,
                        'flux_norm': flux_norm,
                        'name': os.path.basename(path)
                    })

        # --- Création de la figure ---
        plt.style.use('dark_background')
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
        
        # --- Sous-graphique 1 : Avant Normalisation (Spectres Bruts) ---
        ax1.plot(spectra_data[0]['wavelength'], spectra_data[0]['flux_raw'], label=f"Spectre 1 (Brut)", alpha=0.9)
        ax1.plot(spectra_data[1]['wavelength'], spectra_data[1]['flux_raw'], label=f"Spectre 2 (Brut)", alpha=0.9)
        ax1.set_title("Avant Normalisation", fontsize=16)
        ax1.set_xlabel("Longueur d'onde (Å)")
        ax1.set_ylabel("Flux (unités arbitraires)")
        ax1.legend()
        ax1.grid(True, linestyle=':', alpha=0.5)

        # --- Sous-graphique 2 : Après Normalisation ---
        ax2.plot(spectra_data[0]['wavelength'], spectra_data[0]['flux_norm'], label=f"Spectre 1 (Normalisé)")
        ax2.plot(spectra_data[1]['wavelength'], spectra_data[1]['flux_norm'], label=f"Spectre 2 (Normalisé)")
        ax2.set_title("Après Normalisation", fontsize=16)
        ax2.set_xlabel("Longueur d'onde (Å)")
        ax2.set_ylabel("Flux Normalisé par la Médiane")
        ax2.axhline(1.0, color='red', linestyle='--', alpha=0.7, label='Niveau du Continuum (y=1.0)')
        ax2.legend()
        ax2.grid(True, linestyle=':', alpha=0.5)
        
        # --- Finalisation et Sauvegarde ---
        fig.suptitle("Impact de la Normalisation sur les Spectres", fontsize=20, y=1.02)
        plt.tight_layout(rect=[0, 0, 1, 0.96]) # Ajuste pour le titre principal
        
        # Sauvegarder l'image dans le bon dossier pour Docusaurus
        output_path = os.path.join(visualizer.paths["PROJECT_ROOT"], "website/static/img/avant_apres_normalisation.png")
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        
        print(f"Figure de normalisation sauvegardée dans : {output_path}")
        plt.show()

    else:
        print("Pas assez de spectres disponibles pour générer la figure (il en faut au moins 2).")
else:
    print("Veuillez d'abord exécuter la cellule de setup pour initialiser 'visualizer'.")

#